Regression with Binary Variables
===
---
* Linear Probability Model
* Probit Regression
* Logit Regression
---

In [1]:
library(tidyverse)

─ Attaching packages ──────────────────── tidyverse 1.2.1 ─
✔ ggplot2 2.2.1     ✔ purrr   0.2.5
✔ tibble  1.4.2     ✔ dplyr   0.7.6
✔ tidyr   0.8.1     ✔ stringr 1.4.0
✔ readr   1.1.1     ✔ forcats 0.3.0
Warning message:
“package ‘dplyr’ was built under R version 3.5.1”Warning message:
“package ‘stringr’ was built under R version 3.5.2”─ Conflicts ───────────────────── tidyverse_conflicts() ─
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [2]:
charity <- read.csv("/Users/tino/Desktop/TA-Econometrics-II/datasets/0507/charity.csv")

In [6]:
library(stargazer)


Please cite as: 

 Hlavac, Marek (2018). stargazer: Well-Formatted Regression and Summary Statistics Tables.
 R package version 5.2.2. https://CRAN.R-project.org/package=stargazer 



---
## 1. Linear Probability Model

In [7]:
LPM <- lm(respond ~ resplast + weekslast + propresp + mailsyear + giftlast, data = charity)
stargazer(LPM, type = "text", title = "LPM") 


LPM
                        Dependent variable:    
                    ---------------------------
                              respond          
-----------------------------------------------
resplast                     0.067***          
                              (0.019)          
                                               
weekslast                    -0.001***         
                             (0.0002)          
                                               
propresp                     0.650***          
                              (0.037)          
                                               
mailsyear                    0.052***          
                              (0.010)          
                                               
giftlast                     0.0001**          
                             (0.00004)         
                                               
Constant                       0.017           
                              (0.03

---
## 2. Probit Regression

In [9]:
probit <- glm(respond ~ resplast + weekslast + propresp + mailsyear + giftlast, family = binomial(link = "probit"), data = charity)
stargazer(probit, type = "text", title = "Probit")

Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”


Probit
                      Dependent variable:    
                  ---------------------------
                            respond          
---------------------------------------------
resplast                    0.127**          
                            (0.057)          
                                             
weekslast                  -0.005***         
                            (0.001)          
                                             
propresp                   1.848***          
                            (0.114)          
                                             
mailsyear                  0.146***          
                            (0.032)          
                                             
giftlast                     0.001           
                            (0.001)          
                                             
Constant                   -1.296***         
                            (0.114)          
                          

---
## 3. Logit Regression

In [17]:
logit <- glm(respond ~ resplast + weekslast + propresp + mailsyear + giftlast, family = binomial(link = "logit"), data = charity)
stargazer(logit, type = "text", title = "Logit")


Logit
                      Dependent variable:    
                  ---------------------------
                            respond          
---------------------------------------------
resplast                    0.192**          
                            (0.094)          
                                             
weekslast                  -0.008***         
                            (0.001)          
                                             
propresp                   3.034***          
                            (0.191)          
                                             
mailsyear                  0.244***          
                            (0.054)          
                                             
giftlast                     0.002           
                            (0.002)          
                                             
Constant                   -2.117***         
                            (0.193)          
                           

In [15]:
stargazer(LPM, probit, logit, type = "text", title = "Binary Depedent Variable")


Binary Depedent Variable
                                  Dependent variable:              
                    -----------------------------------------------
                                        respond                    
                               OLS              probit    logistic 
                               (1)               (2)        (3)    
-------------------------------------------------------------------
resplast                    0.067***           0.127**    0.192**  
                             (0.019)           (0.057)    (0.094)  
                                                                   
weekslast                   -0.001***         -0.005***  -0.008*** 
                            (0.0002)           (0.001)    (0.001)  
                                                                   
propresp                    0.650***           1.848***   3.034*** 
                             (0.037)           (0.114)    (0.191)  
                      

In [19]:
library(car)
linearHypothesis(logit, c("resplast = 0", "giftlast = 0"))

Loading required package: carData

Attaching package: ‘car’

The following object is masked from ‘package:dplyr’:

    recode

The following object is masked from ‘package:purrr’:

    some



Res.Df,Df,Chisq,Pr(>Chisq)
4264,NA,NA,NA
4262,2,5.804156,0.054909


In [20]:
logit <- glm(respond ~ resplast + weekslast + propresp + mailsyear + giftlast, family = binomial(link = "logit"), data = charity)
logit_H0 <- glm(respond ~ weekslast + propresp + mailsyear, family = binomial(link = "logit"), data = charity)

In [23]:
library(lmtest)
lrtest(logit, logit_H0)

Loading required package: zoo

Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric



#Df,LogLik,Df,Chisq,Pr(>Chisq)
6,-2378.337,NA,NA,NA
4,-2383.267,-2,9.861033,0.007222773
